<a href="https://www.kaggle.com/code/averma111/pytorch-lstm?scriptVersionId=133222683" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [7]:
%%capture
!pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [8]:
%%capture
!pip install torchviz

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [9]:
import torch
import torch.nn.functional as F
from torchviz import make_dot
